In [1]:
!export YOLOX_DATADIR=/efs/data/coco_data/
%env YOLOX_DATADIR=/efs/data/coco_data/


env: YOLOX_DATADIR=/efs/data/coco_data/


In [2]:
!echo $YOLOX_DATADIR

/efs/data/coco_data/


In [3]:
model = "yolox-s"
num_accelerators = 1
total_batch_size = 1
num_workers = 1

In [4]:

env_var_options = "NEURON_RT_ASYNC_EXEC_MAX_INFLIGHT_REQUESTS=3  " + \
    "NEURON_CC_FLAGS=\'--cache_dir=./compiler_cache --model-type=cnn-training\' " + \
    "XLA_IR_DEBUG=1 XLA_HLO_DEBUG=1"

In [ ]:
%env XLA_IR_DEBUG=1
%env XLA_HLO_DEBUG=1

In [1]:
!neuron_parallel_compile

2024-08-29 13:27:01.000593:  311089  INFO ||NEURON_PARALLEL_COMPILE||: Removing existing workdir /tmp/ubuntu/parallel_compile_workdir
usage: neuron_parallel_compile [-h]
                               [--command {collect-and-compile,collect,clear-locks,clear-neffs,compile,clean,scan,scan-with-failed,analyze}]
                               [--num_parallel NUM_PARALLEL]
                               [--node_id NODE_ID] [--world_size WORLD_SIZE]
                               [--static_schedule]
                               [--analyze-output ANALYZE_OUTPUT]
                               [--analyze-verbosity {1,2}]
                               [--parallel_compile_workdir PARALLEL_COMPILE_WORKDIR]
                               [--log_level {INFO,DEBUG,ERROR,WARNING}]

neuron_parallel_compile is an utility to extract graphs from trial run of your script, perform parallel compilation of the graphs, and populate the persistent cache with compiled graphs. Your trial run should be limite

In [6]:
%%time
import subprocess
print("Compile model")
COMPILE_CMD = f"""{env_var_options} neuron_parallel_compile --log_level ERROR --num_parallel 1 torchrun --nproc_per_node={num_workers} -m \
   yolox.tools.train \
    -n {model} \
    -d {num_accelerators} \
    -b {total_batch_size} \
    """

print(f'Running command: \n{COMPILE_CMD}')
if subprocess.check_call(COMPILE_CMD,shell=True):
   print("There was an error with the compilation command")
else:
   print("Compilation Success!!!")

Compile model
Running command: 
NEURON_RT_ASYNC_EXEC_MAX_INFLIGHT_REQUESTS=3  NEURON_CC_FLAGS='--cache_dir=./compiler_cache --model-type=cnn-training' XLA_IR_DEBUG=1 XLA_HLO_DEBUG=1 NEURON_USE_EAGER_DEBUG_MODE=1 neuron_parallel_compile --log_level ERROR --num_parallel 1 torchrun --nproc_per_node=1 -m    yolox.tools.train     -n yolox-s     -d 1     -b 1     
2024-08-29 20:14:26.000363:  341195  INFO ||NEURON_PARALLEL_COMPILE||: Removing existing workdir /tmp/ubuntu/parallel_compile_workdir
2024-08-29 20:14:26.000364:  341195  INFO ||NEURON_PARALLEL_COMPILE||: Running trial run (add option to terminate trial run early; also ignore trial run's generated outputs, i.e. loss, checkpoints)
2024-08-29 20:14:26.000364:  341195  INFO ||NEURON_PARALLEL_COMPILE||: Running cmd: ['torchrun', '--nproc_per_node=1', '-m', 'yolox.tools.train', '-n', 'yolox-s', '-d', '1', '-b', '1']
/bin/sh: 1: nvidia-smi: not found
.2024-08-29 20:14:38 | INFO     | yolox.core.trainer:148 - args: Namespace(experiment_na

concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 246, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/home/ubuntu/aws-neuron-samples/aws_neuron_venv_pytorch/lib/python3.10/site-packages/torch_neuronx/parallel_compile/neuron_parallel_compile.py", line 169, in compile_task
    compile_task_helper(compiled_hlo_status, compile_cache, hlos, workdir, dump=dump,
  File "/home/ubuntu/aws-neuron-samples/aws_neuron_venv_pytorch/lib/python3.10/site-packages/torch_neuronx/parallel_compile/neuron_parallel_compile.py", line 100, in compile_task_helper
    status, retry = libneuronxla.neuron_cc_wrapper.compile_with_cache(
  File "/home/ubuntu/aws-neuron-samples/aws_neuron_venv_pytorch/lib/python3.10/site-packages/libneuronxla/neuron_cc_wrapper.py", line 250, in compile_with_cache
    raise (e)
  File "/home/ubuntu/aws-neuron-samples/aws_neuron_venv_pytorch/lib/pyt

...............

In [6]:
%%time
import subprocess
print("Train model")
RUN_CMD = f"""
   {env_var_options} torchrun --nproc_per_node={num_workers} -m \
   yolox.tools.train \
    -n {model} \
    -d {num_accelerators} \
    -b {total_batch_size} \
    """

print(f'Running command: \n{RUN_CMD}')
if subprocess.check_call(RUN_CMD,shell=True):
   print("There was an error with the fine-tune command")
else:
   print("Fine-tune Successful!!!")

Train model
Running command: 

   NEURON_RT_ASYNC_EXEC_MAX_INFLIGHT_REQUESTS=3  NEURON_CC_FLAGS='--cache_dir=./compiler_cache --model-type=cnn-training' XLA_IR_DEBUG=1 XLA_HLO_DEBUG=1 torchrun --nproc_per_node=1 -m    yolox.tools.train     -n yolox-s     -d 1     -b 1     


/bin/sh: 1: nvidia-smi: not found
2024-08-30 15:43:27 | INFO     | yolox.core.trainer:148 - args: Namespace(experiment_name='yolox_s', name='yolox-s', dist_backend='nccl', dist_url=None, batch_size=1, devices=1, exp_file=None, resume=False, ckpt=None, start_epoch=None, num_machines=1, machine_rank=0, fp16=False, cache=None, occupy=False, logger='tensorboard', opts=[])
2024-08-30 15:43:27 | INFO     | yolox.core.trainer:149 - exp value:
╒═══════════════════╤════════════════════════════╕
│ keys              │ values                     │
╞═══════════════════╪════════════════════════════╡
│ seed              │ None                       │
├───────────────────┼────────────────────────────┤
│ output_dir        │ './YOLOX_outputs'          │
├───────────────────┼────────────────────────────┤
│ print_interval    │ 10                         │
├───────────────────┼────────────────────────────┤
│ eval_interval     │ 10                         │
├───────────────────┼────────────────────────────┤

------------------------ FINISHED before_iter(), PRINTING METRICS

Metric: IrValueTensorToXlaData
  TotalSamples: 388
  Accumulator: 017ms905.105us
  ValueRate: 565ms458.535us / second
  Rate: 12978.2 / second
  Percentiles: 1%=012.061us; 5%=012.303us; 10%=012.453us; 20%=012.733us; 50%=013.473us; 80%=020.730us; 90%=067.495us; 95%=136.617us; 99%=001ms032.919us
Metric: TensorToData
  TotalSamples: 388
  Accumulator: 017ms573.940us
  ValueRate: 554ms352.032us / second
  Rate: 12977.5 / second
  Percentiles: 1%=011.355us; 5%=011.604us; 10%=011.698us; 20%=012.002us; 50%=012.698us; 80%=019.705us; 90%=066.582us; 95%=135.625us; 99%=001ms031.573us
Metric: UnwrapXlaData
  TotalSamples: 776
  Accumulator: 041.759us
  ValueRate: 001ms396.941us / second
  Rate: 25959.1 / second
  Percentiles: 1%=000.038us; 5%=000.041us; 10%=000.044us; 20%=000.046us; 50%=000.050us; 80%=000.053us; 90%=000.059us; 95%=000.096us; 99%=000.148us
Metric: WrapXlaData
  TotalSamples: 388
  Accumulator: 247.371us
  ValueRate:

In [ ]:
import torch_xla.distributed.xla_backend
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch

In [ ]:
gt_classes = torch.tensor([41., 50.], device='xla:0')
gt_classes

In [ ]:
gt_classes.norm()

In [ ]:
gt_classes.max()

In [ ]:
import torch.nn.functional as F
gt_classes = torch.tensor([ 0.,  0., 14., 56., 60.])
num_classes = 80

In [ ]:
gt_cls_per_image = (
    F.one_hot((gt_classes).to(torch.int64), num_classes)
    .float()
)